## Class definitions:

In [2]:
import numpy as np
import copy
class Inv():
    def __init__(self,gamma=1,beta=3):   #metodo costruttore
        self.gamma=gamma
        self.beta=beta
    def p(self):
        return 1
    def g(self):
        return 1
    def h(self,cl):
        return cl/self.cin()
    def cin(self):
        return (self.beta+1)*self.gamma
    def area(self):
      return self.gamma*(1+self.beta)

class Nand():
    def __init__(self,gamma=1,beta=3):   #metodo costruttore
        self.gamma=gamma
        self.beta=beta
    def p(self):
        return (2*self.beta+2)/(self.beta+1)
    def g(self):
        return (self.beta+2)/(self.beta+1)
    def h(self,cl):
        return cl/self.cin()
    def cin(self):
        return (self.beta+2)*self.gamma
    def area(self):
      return self.gamma*(4+2*self.beta)

class Nor():
    def __init__(self,gamma=1,beta=3):   #metodo costruttore
        self.gamma=gamma
        self.beta=beta
    def p(self):
        return (2*self.beta+2)/(self.beta+1)
    def g(self):
        return (2*self.beta+1)/(self.beta+1)
    def h(self,cl):
        return cl/self.cin()
    def cin(self):
        return (2*self.beta+1)*self.gamma
    def area(self):
      return self.gamma*(2+4*self.beta)

class Circuit():
    def __init__(self,gates,branches,cl,cin, tao1, tao2):   #metodo costruttore
        self.gates=gates
        self.cl=cl
        self.cin=cin
        self.branches=branches  #list of number of branches

    def delay_min(self):   #for estimating the minimun delay obtainable
        p_sum=0
        for i in range (len(self.gates)):
            p_sum=p_sum+self.gates[i].p()
        return p_sum+(len(self.gates)*self.f_hat())

    def delay(self):   # for estimating actual delay with real sizes (do resizing before)
        delay=0
        cl=self.cl
        for level,gate in reversed(list(enumerate(self.gates))):
            delay=delay+(gate.p()+gate.g()*gate.h(cl=cl)*self.branches[level])
            cl=gate.cin()
        return delay

    def delay_ps(self):   # for estimating actual delay with real sizes (do resizing before)
      p_tot = 0
      gh_tot = 0
      cl=self.cl
      for level,gate in reversed(list(enumerate(self.gates))):
          p_tot = p_tot+ gate.p()
          gh_tot = gh_tot+ gate.g()*gate.h(cl=cl)*self.branches[level]
          cl=gate.cin()
      delay_ps = p_tot*tao1 + gh_tot*tao2
      return delay_ps

    def B(self):
        B=1
        for b in self.branches:
            B=b*B
        return B

    def G(self):
        G=1
        for gate in self.gates:
            G=gate.g()*G
        return G

    def H(self):
        H=1
        for i, gate in enumerate(self.gates):
            if (i==len(self.gates)-1):
              H=gate.h( cl = self.cl ) * H
            else:
              H=gate.h( cl = self.gates[i+1].cin() ) * H
        return H

    def f_hat(self):
        return (self.G()*self.H()*self.B())**(1/len(self.gates))

    def optimum_sizes(self):    #gives back a list of the optimum sizes of the circuit
        gammas=np.zeros(len(self.gates))
        gamma1=self.gates[0].gamma
        g_prod=1
        for i in range (len(self.gates)):
            if (i==0) :
                gammas[i]=round(gamma1)
            else :
                gammas[i]=round(self.f_hat()*gammas[i-1]/(self.gates[i].g()*self.branches[i-1]))
                if (gammas[i]<1):
                  gammas[i]=1
        return gammas

    def resize(self,gammas):         #taken a list of sizes changes the sizes of all the gates in the circuits
        for i, gate in enumerate (self.gates):
            gate.gamma=gammas[i]

    def deriv_dmin(self,n):
      x=self.H()**(1/n)
      return -x*np.log(x)+x+1

    def circuit_area(self):   #estimate of the area occupied by the trasistors normalized by the minimum size nmos (resize fist)
      area=0
      b=1
      for i, gate in enumerate (self.gates):
        if (i!=0):
          b=b*branches[i-1]
          area=area+gate.area()
        else:
          area=area+gate.area()
      return area

    def n_buffers(self):          #iterative procedure to esteem how many buffer to add for minimizing the delay
      n=2
      sizes=self.optimum_sizes
      self.resize(self.optimum_sizes())
      previous_delay=self.delay_ps()
      delay=0
      a=0
      while(a==0):
        circuit_iteration=copy.deepcopy(self)
        for _ in range(n):
          circuit_iteration.gates.append(Inv())
          circuit_iteration.branches.append(1)
        circuit_iteration.resize(circuit_iteration.optimum_sizes())
        delay=circuit_iteration.delay_ps()
        if(delay<previous_delay):
          a=0
          n=n*2
          previous_delay=delay
        else:
          a=1
          if(n==2):
            n=0
          else:
            n=n/2
      a=0
      while((a==0) and (n>0)):
        circuit_iteration.gates.pop()
        circuit_iteration.gates.pop()
        circuit_iteration.branches.pop()
        circuit_iteration.branches.pop()
        circuit_iteration.resize(circuit_iteration.optimum_sizes())
        delay=circuit_iteration.delay_ps()
        if(delay<previous_delay):
          n=n-2
          a=0
        else:
          a=1
          #n=n+2
        previous_delay=delay
      return n/2

    def reasonable_sizes(self, tolerance=0.1):
      sizes=self.optimum_sizes()
      self.resize(sizes)
      initial_delay=self.delay()
      a=0
      while (a==0):
        for i in range(len(sizes)):
          if (sizes[i]>(1/0.95)):
            sizes[i]=round(sizes[i]*0.95)
        self.resize(sizes)
        delay=self.delay()
        if(delay>initial_delay*(1+tolerance)):
          a=1
      return sizes


$C_{WL}$ & $C_{wire}$  estimation results

In [5]:
Cwl=25   #fF
Cwire=38     #fF
CinInv= 1.1 #fF
tao1 = 16.51 #ps
tao2 = 4.974 #ps

### Nand 1x configuration

In [6]:
beta=3
gates=[Inv(),Inv(),Inv(),Nand(),Nor(),Nand(),Inv()]  #wire fra nor e nand
branches=[1,1,8,1,16,1,1]
# post decoder:
right_side=Circuit(gates=[Nand(beta=beta),Nor(beta=beta),Nand(beta=beta),Inv(beta=beta)],branches=[1,1,1,1], cl= Cwl/CinInv  , cin=1, tao1= tao1, tao2= tao2)
print(f"Post-decoder optimum sizes: {right_side.optimum_sizes()}")
right_side.resize(right_side.optimum_sizes())
print(f"Post-decoder delay: {right_side.delay_min()}")
print(f"Post-decoder delay ps: {right_side.delay_ps()}")
print(f"Post-decoder area: {right_side.circuit_area()}")
print()
#print(right_side.f_hat())
# pre decoder:
left_side=Circuit(gates=[Inv(beta=beta),Inv(beta=beta),Inv(beta=beta)], branches=[1,1,1], cl= 128*(Cwire/CinInv + 4/3)  , cin=1, tao1= tao1, tao2= tao2)
print("Pre-decoder optimum sizes: ",left_side.optimum_sizes())
sizes=left_side.optimum_sizes()
left_side.resize(sizes)
print("Pre-decoder delay [no buffering, no tao]: ",left_side.delay_min())
print(f"Pre-decoder delay ps: {left_side.delay_ps()}")
print("Pre-decoder area: ",left_side.circuit_area())
print()

total_delay_noReasonable = right_side.delay()+left_side.delay()
total_delay_noReasonable_ps = right_side.delay_ps()+left_side.delay_ps()
print(f"Total delay [no reasonable, no tao]: {total_delay_noReasonable}")
print(f"Total delay ps: {total_delay_noReasonable_ps}")
total_area_noReasonable = 16*left_side.circuit_area()+256*right_side.circuit_area()
print(f"Total area [no reasonable]: {total_area_noReasonable}")
print()
#print(left_side.f_hat())
"""
# buffering estimation
n_buffers=left_side.n_buffers()
print("Buffers to add: ",n_buffers)
for _ in range(int(n_buffers)*2):
  left_side.gates.append(Inv())
  left_side.branches.append(1)
print(len(left_side.gates))
print(left_side.optimum_sizes())
print("Left side delay: after buffering",left_side.delay_min())
"""

reasonable_sizes=left_side.reasonable_sizes()
print("Pre-decoder sizes [reasonable]: ",reasonable_sizes)
left_side.resize(reasonable_sizes)
print("Pre-decoder delay [reasonable]:", left_side.delay_ps(),"ps")
print("Pre-decoder area [reasonable]: ",left_side.circuit_area())
print()
total_delay_Reasonable = right_side.delay_ps()+left_side.delay_ps()
print(f"Total delay [reasonable]: {total_delay_Reasonable}")
total_area_Reasonable = 16*left_side.circuit_area()+ 256*right_side.circuit_area()
print(f"Total area [reasonable]: {total_area_Reasonable}")
print()

print(f"Delay increase: {100*(total_delay_Reasonable/total_delay_noReasonable)-100}%")
print(f"Area reduction: {100-(total_area_Reasonable/total_area_noReasonable)*100}%")

Post-decoder optimum sizes: [1. 1. 2. 4.]
Post-decoder delay: 14.510499815709778
Post-decoder delay ps: 153.7228409090909
Post-decoder area: 60.0

Pre-decoder optimum sizes:  [  1.  10. 105.]
Pre-decoder delay [no buffering, no tao]:  34.41356086172673
Pre-decoder delay ps: 205.88514199134198
Pre-decoder area:  464.0

Total delay [no reasonable, no tao]: 49.10494227994228
Total delay ps: 359.6079829004329
Total area [no reasonable]: 22784.0

Pre-decoder sizes [reasonable]:  [ 1. 10. 60.]
Pre-decoder delay [reasonable]: 224.29324848484848 ps
Pre-decoder area [reasonable]:  284.0

Total delay [reasonable]: 378.0160893939394
Total area [reasonable]: 19904.0

Delay increase: 669.8127150601423%
Area reduction: 12.640449438202253%


### Nor 1x configuration:

In [7]:
beta=3
gates=[Inv(),Inv(),Inv(),Nand(),Nor(),Nand(),Inv()]  #wire fra nor e nand
branches=[1,1,8,1,16,1,1]
# post decoder:
right_side=Circuit(gates=[Nor(),Nand(),Nor()],branches=[1,1,1], cl= Cwl/CinInv  , cin=1, tao1= tao1, tao2= tao2)
print(f"Post-decoder optimum sizes: {right_side.optimum_sizes()}")
right_side.resize(right_side.optimum_sizes())
print(f"Post-decoder delay: {right_side.delay_min()}")
print(f"Post-decoder delay ps: {right_side.delay_ps()}")
print(f"Post-decoder area: {right_side.circuit_area()}")
print()
#print(right_side.f_hat())
# pre decoder:
left_side=Circuit(gates=[Inv(),Inv(),Inv()], branches=[1,1,1], cl= 128*(Cwire/CinInv + 5/3)  , cin=1, tao1= tao1, tao2= tao2)
print("Pre-decoder optimum sizes: ",left_side.optimum_sizes())
sizes=left_side.optimum_sizes()
left_side.resize(sizes)
print("Pre-decoder delay [no buffering, no tao]: ",left_side.delay_min())
print(f"Pre-decoder delay ps: {left_side.delay_ps()}")
print("Pre-decoder area: ",left_side.circuit_area())
print()

total_delay_noReasonable = right_side.delay()+left_side.delay()
total_delay_noReasonable_ps = right_side.delay_ps()+left_side.delay_ps()
print(f"Total delay [no reasonable, no tao]: {total_delay_noReasonable}")
print(f"Total delay ps: {total_delay_noReasonable_ps}")
total_area_noReasonable = 16*left_side.circuit_area()+256*right_side.circuit_area()
print(f"Total area [no reasonable]: {total_area_noReasonable}")
print()
#print(left_side.f_hat())
"""
# buffering estimation
n_buffers=left_side.n_buffers()
print("Buffers to add: ",n_buffers)
for _ in range(int(n_buffers)*2):
  left_side.gates.append(Inv())
  left_side.branches.append(1)
print(len(left_side.gates))
print(left_side.optimum_sizes())
print("Left side delay: after buffering",left_side.delay_min())
"""

reasonable_sizes=left_side.reasonable_sizes()
print("Pre-decoder sizes [reasonable]: ",reasonable_sizes)
left_side.resize(reasonable_sizes)
print("Pre-decoder delay [reasonable]:", left_side.delay())
print("Pre-decoder area [reasonable]: ",left_side.circuit_area())
print()
total_delay_Reasonable = right_side.delay()+left_side.delay()
print(f"Total delay [reasonable]: {total_delay_Reasonable}")
total_area_Reasonable = 16*left_side.circuit_area()+ 256*right_side.circuit_area()
print(f"Total area [reasonable]: {total_area_Reasonable}")
print()

print(f"Delay increase: {100*(total_delay_Reasonable/total_delay_noReasonable)-100}%")
print(f"Area reduction: {100-(total_area_Reasonable/total_area_noReasonable)*100}%")

Post-decoder optimum sizes: [1. 2. 3.]
Post-decoder delay: 12.949171865259736
Post-decoder delay ps: 133.97220454545456
Post-decoder area: 76.0

Pre-decoder optimum sizes:  [  1.  11. 116.]
Pre-decoder delay [no buffering, no tao]:  34.510544123533485
Pre-decoder delay ps: 206.3851159874608
Pre-decoder area:  512.0

Total delay [no reasonable, no tao]: 47.553944618599786
Total delay ps: 340.35732053291537
Total area [no reasonable]: 27648.0

Pre-decoder sizes [reasonable]:  [ 1. 10. 60.]
Pre-decoder delay [reasonable]: 38.31313131313131
Pre-decoder area [reasonable]:  284.0

Total delay [reasonable]: 51.3320707070707
Total area [reasonable]: 24000.0

Delay increase: 7.944926795816571%
Area reduction: 13.194444444444443%


### Nand 2x configuration:

In [8]:
beta=3
# post decoder:
right_side=Circuit(gates=[Nor(beta=beta),Nand(beta=beta),Inv(beta=beta)],branches=[1,1,1], cl= Cwl/CinInv  , cin=1, tao1= tao1, tao2= tao2)
print(f"Post-decoder optimum sizes: {right_side.optimum_sizes()}")
right_side.resize(right_side.optimum_sizes())
print(f"Post-decoder delay: {right_side.delay_min()}")
print(f"Post-decoder delay ps: {right_side.delay_ps()}")
print(f"Post-decoder area: {right_side.circuit_area()}")
print()
#print(right_side.f_hat())
# pre decoder:
left_side=Circuit(gates=[Inv(beta=beta),Inv(beta=beta),Inv(beta=beta),Nand(beta=beta)], branches=[1,1,2,1], cl= 64*(Cwire/CinInv + 5/3)  , cin=1, tao1= tao1, tao2= tao2)
print("Pre-decoder optimum sizes: ",left_side.optimum_sizes())
sizes=left_side.optimum_sizes()
left_side.resize(sizes)
print("Pre-decoder delay [no buffering, no tao]: ",left_side.delay_min())
print(f"Pre-decoder delay ps: {left_side.delay_ps()}")
print("Pre-decoder area: ",left_side.circuit_area())
print()

total_delay_noReasonable = right_side.delay()+left_side.delay()
total_delay_noReasonable_ps = right_side.delay_ps()+left_side.delay_ps()
print(f"Total delay [no reasonable, no tao]: {total_delay_noReasonable}")
print(f"Total delay ps: {total_delay_noReasonable_ps}")
total_area_noReasonable = 16*left_side.circuit_area()+256*right_side.circuit_area()
print(f"Total area [no reasonable]: {total_area_noReasonable}")
print()
#print(left_side.f_hat())
"""
# buffering estimation
n_buffers=left_side.n_buffers()
print("Buffers to add: ",n_buffers)
for _ in range(int(n_buffers)*2):
  left_side.gates.append(Inv())
  left_side.branches.append(1)
print(len(left_side.gates))
print(left_side.optimum_sizes())
print("Left side delay: after buffering",left_side.delay_min())
"""

reasonable_sizes=left_side.reasonable_sizes()
print("Pre-decoder sizes [reasonable]: ",reasonable_sizes)
left_side.resize(reasonable_sizes)
print("Pre-decoder delay [reasonable]:", left_side.delay())
print("Pre-decoder area [reasonable]: ",left_side.circuit_area())
print()
total_delay_Reasonable = right_side.delay()+left_side.delay()
print(f"Total delay [reasonable]: {total_delay_Reasonable}")
total_area_Reasonable = 16*left_side.circuit_area()+ 256*right_side.circuit_area()
print(f"Total area [reasonable]: {total_area_Reasonable}")
print()

print(f"Delay increase: {100*(total_delay_Reasonable/total_delay_noReasonable)-100}%")
print(f"Area reduction: {100-(total_area_Reasonable/total_area_noReasonable)*100}%")

Post-decoder optimum sizes: [1. 2. 4.]
Post-decoder delay: 10.76660719871472
Post-decoder delay ps: 111.99834090909093
Post-decoder area: 50.0

Pre-decoder optimum sizes:  [ 1.  6. 37. 91.]
Pre-decoder delay [no buffering, no tao]:  29.676779503111586
Pre-decoder delay ps: 205.3196690876691
Pre-decoder area:  1086.0

Total delay [no reasonable, no tao]: 40.60273622773623
Total delay ps: 317.31800999676
Total area [no reasonable]: 30176.0

Pre-decoder sizes [reasonable]:  [ 1.  6. 20. 46.]
Pre-decoder delay [reasonable]: 32.67885375494071
Pre-decoder area [reasonable]:  568.0

Total delay [reasonable]: 43.59930830039526
Total area [reasonable]: 21888.0

Delay increase: 7.380222002407891%
Area reduction: 27.46553552492047%


### Nor 2x configuration:

In [9]:
beta=3
# post decoder:
right_side=Circuit(gates=[Nand(),Nor()],branches=[1,1], cl= Cwl/CinInv  , cin=1, tao1= tao1, tao2= tao2)
print(f"Post-decoder optimum sizes: {right_side.optimum_sizes()}")
right_side.resize(right_side.optimum_sizes())
print(f"Post-decoder delay: {right_side.delay_min()}")
print(f"Post-decoder delay ps: {right_side.delay_ps()}")
print(f"Post-decoder area: {right_side.circuit_area()}")
print()
#print(right_side.f_hat())
# pre decoder:
left_side=Circuit(gates=[Inv(),Inv(),Inv(),Nor()], branches=[1,1,2,1], cl= 64*(Cwire/CinInv + 4/3)  , cin=1, tao1= tao1, tao2= tao2)
print("Pre-decoder optimum sizes: ",left_side.optimum_sizes())
sizes=left_side.optimum_sizes()
left_side.resize(sizes)
print("Pre-decoder delay [no buffering, no tao]: ",left_side.delay_min())
print(f"Pre-decoder delay ps: {left_side.delay_ps()}")
print("Pre-decoder area: ",left_side.circuit_area())
print()

total_delay_noReasonable = right_side.delay()+left_side.delay()
total_delay_noReasonable_ps = right_side.delay_ps()+left_side.delay_ps()
print(f"Total delay [no reasonable, no tao]: {total_delay_noReasonable}")
print(f"Total delay ps: {total_delay_noReasonable_ps}")
total_area_noReasonable = 16*left_side.circuit_area()+256*right_side.circuit_area()
print(f"Total area [no reasonable]: {total_area_noReasonable}")
print()
#print(left_side.f_hat())
"""
# buffering estimation
n_buffers=left_side.n_buffers()
print("Buffers to add: ",n_buffers)
for _ in range(int(n_buffers)*2):
  left_side.gates.append(Inv())
  left_side.branches.append(1)
print(len(left_side.gates))
print(left_side.optimum_sizes())
print("Left side delay: after buffering",left_side.delay_min())
"""

reasonable_sizes=left_side.reasonable_sizes()
print("Pre-decoder sizes [reasonable]: ",reasonable_sizes)
left_side.resize(reasonable_sizes)
print("Pre-decoder delay [reasonable]:", left_side.delay())
print("Pre-decoder area [reasonable]: ",left_side.circuit_area())
print()
total_delay_Reasonable = right_side.delay()+left_side.delay()
print(f"Total delay [reasonable]: {total_delay_Reasonable}")
total_area_Reasonable = 16*left_side.circuit_area()+ 256*right_side.circuit_area()
print(f"Total area [reasonable]: {total_area_Reasonable}")
print()

print(f"Delay increase: {100*(total_delay_Reasonable/total_delay_noReasonable)-100}%")
print(f"Area reduction: {100-(total_area_Reasonable/total_area_noReasonable)*100}%")

Post-decoder optimum sizes: [1. 2.]
Post-decoder delay: 10.306562238868912
Post-decoder delay ps: 97.57968181818183
Post-decoder area: 38.0

Pre-decoder optimum sizes:  [ 1.  7. 47. 90.]
Pre-decoder delay [no buffering, no tao]:  31.780361720840695
Pre-decoder delay ps: 215.82765628319672
Pre-decoder area:  1480.0

Total delay [no reasonable, no tao]: 42.135773643220446
Total delay ps: 313.4073381013785
Total area [no reasonable]: 33408.0

Pre-decoder sizes [reasonable]:  [ 1.  7. 23. 42.]
Pre-decoder delay [reasonable]: 35.345128301650035
Pre-decoder area [reasonable]:  712.0

Total delay [reasonable]: 45.68603739255913
Total area [reasonable]: 21120.0

Delay increase: 8.425770888651797%
Area reduction: 36.7816091954023%


### Nand 4x configuration

In [14]:
beta=3
gates=[Inv(),Inv(),Inv(),Nand(),Nor(),Nand(),Inv()]  #wire fra nor e nand
branches=[1,1,8,1,16,1,1]
# post decoder:
right_side=Circuit(gates=[Nand(beta=beta),Inv(beta=beta)],branches=[1,1], cl= Cwl/CinInv  , cin=1, tao1= tao1, tao2= tao2)
print(f"Post-decoder optimum sizes: {right_side.optimum_sizes()}")
right_side.resize(right_side.optimum_sizes())
print(f"Post-decoder delay: {right_side.delay()}")
print(f"Post-decoder delay ps: {right_side.delay_ps()}")
print(f"Post-decoder area: {right_side.circuit_area()}")
print()
#print(right_side.f_hat())
# pre decoder:
left_side=Circuit(gates=[Inv(beta=beta),Inv(beta=beta),Inv(beta=beta),Nand(beta=beta),Nor(beta=beta)], branches=[1,1,8,1,1], cl= 16*(Cwire/CinInv + 4/3)  , cin=1, tao1= tao1, tao2= tao2)
print("Pre-decoder optimum sizes: ",left_side.optimum_sizes())
sizes=left_side.optimum_sizes()
left_side.resize(sizes)
print("Pre-decoder delay [no buffering, no tao]: ",left_side.delay_min())
print(f"Pre-decoder delay ps: {left_side.delay_ps()}")
print("Pre-decoder area: ",left_side.circuit_area())
print()

total_delay_noReasonable = right_side.delay()+left_side.delay()
total_delay_noReasonable_ps = right_side.delay_ps()+left_side.delay_ps()
print(f"Total delay [no reasonable, no tao]: {total_delay_noReasonable}")
print(f"Total delay ps: {total_delay_noReasonable_ps}")
total_area_noReasonable = 16*left_side.circuit_area()+256*right_side.circuit_area()
print(f"Total area [no reasonable]: {total_area_noReasonable}")
print()
#print(left_side.f_hat())

# buffering estimation
print("Before buffering:")
print("Pre-decoder optimum sizes: ",left_side.optimum_sizes())
print(f"Pre-decoder delay: {round(left_side.delay_ps())} ps")
print("Pre-decoder area: ",left_side.circuit_area())
print()
print("After buffering:")
n_buffers=1
print("Buffers added: ",n_buffers)
for _ in range(int(n_buffers)*2):
  left_side.gates.append(Inv())
  left_side.branches.append(1)
#print(len(left_side.gates))
print(f"New sizing of pre-decoder gates: {left_side.optimum_sizes()}")
left_side.resize(left_side.optimum_sizes())
print(f"New pre-decoder delay: {round(left_side.delay_ps())} ps")
print("New pre-decoder area: ",left_side.circuit_area())
print()

reasonable_sizes=left_side.reasonable_sizes()
print("Pre-decoder sizes [reasonable]: ",reasonable_sizes)
left_side.resize(reasonable_sizes)
print("Pre-decoder delay [reasonable]:", left_side.delay())
print("Pre-decoder area [reasonable]: ",left_side.circuit_area())
print()
total_delay_Reasonable = right_side.delay()+left_side.delay()
total_delay_Reasonable_ps = right_side.delay_ps()+left_side.delay_ps()
print(f"Total delay [reasonable]: {total_delay_Reasonable}")
print(f"Total delay [reasonable]: {total_delay_Reasonable_ps} ps")
total_area_Reasonable = 16*left_side.circuit_area()+256*right_side.circuit_area()
print(f"Total area [reasonable]: {total_area_Reasonable}")
print()

print(f"Delay increase: {100*(total_delay_Reasonable/total_delay_noReasonable)-100}%")
print(f"Area reduction: {100-(total_area_Reasonable/total_area_noReasonable)*100}%")

Post-decoder optimum sizes: [1. 2.]
Post-decoder delay: 7.840909090909091
Post-decoder delay ps: 73.60868181818182
Post-decoder area: 18.0

Pre-decoder optimum sizes:  [ 1.  5. 24. 11. 30.]
Pre-decoder delay [no buffering, no tao]:  30.930797114855842
Pre-decoder delay ps: 234.64705757575757
Pre-decoder area:  650.0

Total delay [no reasonable, no tao]: 38.78080808080808
Total delay ps: 308.2557393939394
Total area [no reasonable]: 15008.0

Before buffering:
Pre-decoder optimum sizes:  [ 1.  5. 24. 11. 30.]
Pre-decoder delay: 235 ps
Pre-decoder area:  650.0

After buffering:
Buffers added:  1
New sizing of pre-decoder gates: [ 1.  3.  9.  3.  5. 15. 46.]
New pre-decoder delay: 255 ps
New pre-decoder area:  396.0

Pre-decoder sizes [reasonable]:  [ 1.  3.  9.  3.  5. 10. 16.]
Pre-decoder delay [reasonable]: 33.81969696969697
Pre-decoder area [reasonable]:  256.0

Total delay [reasonable]: 41.660606060606064
Total delay [reasonable]: 345.6518545454545 ps
Total area [reasonable]: 8704.0



In [11]:
print("Base version of Logical Effort Model:")
print("Pre-decoder optimum sizes:",left_side.optimum_sizes())
print(f"Total area found: {total_area_noReasonable}")
print(f"Total delay: {round(total_delay_noReasonable_ps)} ps")
print()
print("Optimization of area with slight increase of delay:")
print("New pre-decoder sizes: ",reasonable_sizes)
print(f"New total area: {total_area_Reasonable}")
print(f"New total delay: {round(total_delay_Reasonable_ps)} ps")
print()
print(f"Delay increase: {round(100*(total_delay_Reasonable/total_delay_noReasonable)-100, 1)}%")
print(f"Area reduction: {round(100-(total_area_Reasonable/total_area_noReasonable)*100, 1)}%")

Base version of Logical Effort Model:
Pre-decoder optimum sizes: [ 1.  5. 24. 11. 30.]
Total area found: 15008.0
Total delay: 308 ps

Optimization of area with slight increase of delay:
New pre-decoder sizes:  [ 1.  5. 12. 10. 17.]
New total area: 11168.0
New total delay: 324 ps

Delay increase: 8.3%
Area reduction: 25.6%


### Nor 4x configuration

In [12]:
beta=3
gates=[Inv(),Inv(),Inv(),Nand(),Nor(),Nand(),Inv()]  #wire fra nor e nand
branches=[1,1,8,1,16,1,1]
# post decoder:
right_side=Circuit(gates=[Nor(beta=beta),Inv(beta=beta)],branches=[1,1], cl= Cwl/CinInv  , cin=1, tao1= tao1, tao2= tao2)
print(f"Post-decoder optimum sizes: {right_side.optimum_sizes()}")
right_side.resize(right_side.optimum_sizes())
print(f"Post-decoder delay: {right_side.delay()}")
print(f"Post-decoder delay ps: {right_side.delay_ps()}")
print(f"Post-decoder area: {right_side.circuit_area()}")
print()
#print(right_side.f_hat())
# pre decoder:
left_side=Circuit(gates=[Inv(beta=beta),Inv(beta=beta),Inv(beta=beta),Nor(beta=beta),Nand(beta=beta)], branches=[1,1,8,1,1], cl= 16*(Cwire/CinInv + 4/3)  , cin=1, tao1= tao1, tao2= tao2)
print("Pre-decoder optimum sizes: ",left_side.optimum_sizes())
sizes=left_side.optimum_sizes()
left_side.resize(sizes)
print("Pre-decoder delay [no buffering, no tao]: ",left_side.delay_min())
print(f"Pre-decoder delay ps: {left_side.delay_ps()}")
print("Pre-decoder area: ",left_side.circuit_area())
print()

total_delay_noReasonable = right_side.delay()+left_side.delay()
total_delay_noReasonable_ps = right_side.delay_ps()+left_side.delay_ps()
print(f"Total delay [no reasonable, no tao]: {total_delay_noReasonable}")
print(f"Total delay ps: {total_delay_noReasonable_ps}")
total_area_noReasonable = 16*left_side.circuit_area()+256*right_side.circuit_area()
print(f"Total area [no reasonable]: {total_area_noReasonable}")
print()
#print(left_side.f_hat())
"""
# buffering estimation
n_buffers=left_side.n_buffers()
print("Buffers to add: ",n_buffers)
for _ in range(int(n_buffers)*2):
  left_side.gates.append(Inv())
  left_side.branches.append(1)
print(len(left_side.gates))
print(left_side.optimum_sizes())
print("Left side delay: after buffering",left_side.delay_min())
"""

reasonable_sizes=left_side.reasonable_sizes()
print("Pre-decoder sizes [reasonable]: ",reasonable_sizes)
left_side.resize(reasonable_sizes)
print("Pre-decoder delay [reasonable]:", left_side.delay())
print("Pre-decoder area [reasonable]: ",left_side.circuit_area())
print()
total_delay_Reasonable = right_side.delay()+left_side.delay()
print(f"Total delay [reasonable]: {total_delay_Reasonable}")
total_area_Reasonable = 16*left_side.circuit_area()+256*right_side.circuit_area()
print(f"Total area [reasonable]: {total_area_Reasonable}")
print()

print(f"Delay increase: {100*(total_delay_Reasonable/total_delay_noReasonable)-100}%")
print(f"Area reduction: {100-(total_area_Reasonable/total_area_noReasonable)*100}%")

Post-decoder optimum sizes: [1. 2.]
Post-decoder delay: 7.840909090909091
Post-decoder delay ps: 73.60868181818182
Post-decoder area: 22.0

Pre-decoder optimum sizes:  [ 1.  5. 24.  8. 31.]
Pre-decoder delay [no buffering, no tao]:  30.930797114855842
Pre-decoder delay ps: 234.6472500366569
Pre-decoder area:  542.0

Total delay [no reasonable, no tao]: 38.78084677419355
Total delay ps: 308.25593185483876
Total area [no reasonable]: 14304.0

Pre-decoder sizes [reasonable]:  [ 1.  5. 12.  8. 18.]
Pre-decoder delay [reasonable]: 34.51889730639731
Pre-decoder area [reasonable]:  364.0

Total delay [reasonable]: 42.3598063973064
Total area [reasonable]: 11456.0

Delay increase: 9.228678383305564%
Area reduction: 19.91051454138703%


#Estimated parasitics

In [13]:
# Bit cell array capacitance
ca_M3 = 0.034  # parallel plate cap
cf_M3 = 0.008  # fringe cap
cc_M3 = 0.072  # coupling cap
r_M3 = 105  # sheet res
widthM3 = 0.1  # um
lengthM3 = 32  # um
dmin = 0.1  # um
Co = 0.36  # fF
widthM2 = 0.100  # um
lengthM2 = 128  # um
ca_M2 = 0.048  # parallel plate cap
cf_M2 = 0.009  # fringe cap
cc_M2 = 0.071  # coupling cap

BCA_cap = ca_M3 * widthM3 * lengthM3 + cf_M3 * lengthM3 + cc_M3 * lengthM3 * 3 * dmin / 0.5  # A factor 3 was considered as the worst case intercoupling capacitance,
bitcell_cap = 32 * Co * 2
total_cap = BCA_cap + bitcell_cap
r_M2 = 105 * lengthM2 / widthM2
r_M3 = 105 * lengthM3 / widthM3

Dec_cap = ca_M2 * widthM2 * lengthM2 + cf_M2 * lengthM2 + cc_M2 * lengthM2 * 4 # A factor 4 was considered as the worst case intercoupling capacitance. Wires are at minimum distance

print("BCA_cap:", round(BCA_cap, 1))
print("bitcell_cap:", round(bitcell_cap, 1))
print("total_cap:", round(total_cap, 1))
print("r_M2:", round(r_M2, 1))
print("r_M3:", round(r_M3, 1))
print("Dec_cap:", round(Dec_cap, 1))


BCA_cap: 1.7
bitcell_cap: 23.0
total_cap: 24.8
r_M2: 134400.0
r_M3: 33600.0
Dec_cap: 38.1
